# Exact approximations of Split-and-lookup S-boxes

## Computing the number of approximations

In [ ]:
# Generating the list of values

def gen_lis(L, a, b):
    """
    Generate a table:
    * keys are the values achieved by the map a*L(x)-b*x
    * values are lists of x's achieving this value
    """
    lis = {}
    
    for x in range(len(L)):
        y = a*L[x] - b*x
        if y in lis.keys():
            lis[y].append(x)
        else:
            lis[y] = [x]
    return lis

In [ ]:
# Working with linear approximation matrices

def get_count(c, lis):
    """
    Get the number of x such that a*L(x)-b*x = c.    
    """
    if c in lis.keys():
        return len(lis[c])
    else:
        return 0

def const_to_mat(c, lis):
    """
    Compute the matrix whose coefficients are
    * c_ij = #{x | a*L(x)-b*x = c+2^8*i - j}
    """
    I_length = ((max(lis.keys())-min(lis.keys())) // 256) +1
    
    mat = matrix(ZZ, [[get_count(c-i+j*256, lis) for i in range(-I_length, I_length+1)]
                      for j in range(-I_length, I_length+1)])
    
    return mat


def list_to_mat(c_list, lis):
    """
    Compute the same matrix, but for a list of constants using matrix product.
    """
    mat = const_to_mat(c_list[0], lis)
    for c in c_list[1:]:
        mat = mat * const_to_mat(c, lis)
    return mat




In [ ]:
# Computing the exact number of approximations
# in the case where the field is the Goldilocks field.

def approximations_goldi(c_list, lis):
    """
    Compute the number of x such that a*S(x)-b*x = c.
    
    We take into account the reduction modulo p.
    """
    I_length = ((max(lis.keys())-min(lis.keys())) // 256) +1
    n_dim = 2*I_length + 1
    score = 0
    
    # Compute left and right matrices
    mat_left  = list_to_mat(c_list[:4], lis)
    mat_right = list_to_mat(c_list[4:], lis)
    
    for i in range(n_dim):
        min_k = max(0, i-I_length)
        max_k = min(n_dim, i+I_length+1)
        
        # For all i, k we have the following:
        # (i*2^32-k)*2^32 + (k-i)*2^32 - i = 0 mod p
        for k in range(min_k, max_k):
            score += mat_left[i, k]*mat_right[k-i+I_length, n_dim -i-1]
    return score

def int_to_list(c):
    """
    Transform a Fp field element into a list of integers
    Decomposition in base 256.
    """
    c_list = []
    
    for i in range(8):
        c_list.append(c % 256)
        c //= 256
    c_list.reverse()
    return c_list

def approximations_goldi_abc(L, a, b, c):
    """
    Compute the number of x such that
    a*S(x) = b*x + c
    """
    lis = gen_lis(L, a, b)
    
    c_list = int_to_list(c)
    score = approximations_goldi(c_list, lis)
    return score
    

## Some examples of Split-and-lookups

### AES S-box (Rescaled)

In [ ]:
from sage.crypto.sboxes import sboxes

F28.<x> = GF(2)[x].quotient(x**8 + x**4 + x**3 + x + 1)

def int_to_F28(xi):
    res = F28(0)
    
    for i in range(8):
        res += (xi % 2) *x**i
        xi//= 2
    return res

def F28_to_int(xi):
    res = 0
    
    for i in range(len(xi.list())):
        res += int(xi.list()[i])*2**i
    return res


In [ ]:
# Get AES S-box
L_aes = sboxes["AES"]

# Add fixed points 0x00 and 0xff
L_aes = [int_to_F28(y ^^ 0x63) for y in L_aes]
L_aes = [y*int_to_F28(0xff)/int_to_F28(0x75) for y in L_aes]
L_aes = [F28_to_int(y) for y in L_aes]

print("AES S-box:")
print(L_aes)

a, b, c = 1, 1, 2025524839466146844

print("Best approximation found: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations_goldi_abc(L_aes, a, b, c)))

### iScream S-box (Rescaled)

In [ ]:
# Get iScream S-box
L_iscream = sboxes["iScream"]

# Add fixed point 0xff
L_iscream = [y ^^ (y << 1) & 0xff for y in L_iscream]
L_iscream = [y ^^ (y << 4) & 0xff for y in L_iscream]

print("iScream S-box:")
print(L_iscream)

a, b, c = 1, -1, 1157442769704194062

print("Best approximation found: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations_goldi_abc(L_iscream, a, b, c)))

### Tip5 S-box

In [ ]:
Fr = GF(257)

In [ ]:
L_tip5 = [(Fr(x)+1)**3-1 for x in range(256)]
L_tip5 = [int(y) for y in L_tip5]

print("Tip5 S-box:")
print(L_tip5)

a, b, c = 1, -1, 606306544323790996

print("Best approximation found: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations_goldi_abc(L_tip5, a, b, c)))

### Monolith S-box

In [ ]:
def shift(x, sh, n):
    return ((x  << sh) | (x >> (n-sh))) & (2**n-1)

def not_(x, n):
    return x ^^ (2**n -1)

def chi_daemen_shift(x):
    return shift(x ^^ (shift(not_(x, 8), 1, 8) & shift(x, 2, 8) & shift(x, 3, 8)), 1, 8)


In [ ]:
L_monolith = [chi_daemen_shift(x) for x in range(256)]

print("Monolith S-box:")
print(L_monolith)

a, b, c = 1, 2, 0

print("Best approximation found: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations_goldi_abc(L_monolith, a, b, c)))

# Walsh spectrum of Split-and-lookups

In [ ]:
p = 2**64-2**32+1

## Computing correlation for a given mask

In [ ]:
def walsh_coeff(L, a, b):
    """
    Compute a coefficient of the Walsh transform of S.
    """
    s     = 8*[0]
    ones  = 8*[0]
    zeros = 8*[0]
    
    zeta_p = CC.zeta(p)
    
    # Compute the coefficients.
    for i in range(8):
        for x in range(256):
            s[i] += zeta_p**((a*L[x]-b*x)*2**(8*i) % p)
        ones[i]   = zeta_p**((a*L[0xff]-b*0xff)*2**(8*i) % p)
        zeros[i]  = zeta_p**((a*L[0x00]-b*0x00)*2**(8*i) % p)
    
    # Compute the sum over x<2^64.
    prod_64 = 1
    for i in range(8):
        prod_64 *= s[i]
    
    # Compute the sum over values counted twice 
    # (ie of the form 0xffffffff********).
    prod_32 = 1
    for i in range(4):
        prod_32 *= s[i]
    for i in range(4,8):
        prod_32 *= ones[i]
    
    # Compute the value for 0xffffffff00000000.
    prod_0 = 1
    for i in range(4):
        prod_0 *= zeros[i]
    for i in range(4,8):
        prod_0 *= ones[i]
    
    res = prod_64 - prod_32 + prod_0
    return res


## Examples of Tip5 and Monolith.

In [ ]:
a, b = 22, 14

print("Correlation for Tip5:")
print("{}*S(x) - {}*x : {}".format(a, b, abs(walsh_coeff(L_tip5, a, b))/p))

In [ ]:
a, b = 1, 2

print("Correlation for Monolith:")
print("{}*S(x) - {}*x : {}".format(a, b, abs(walsh_coeff(L_monolith, a, b))/p))